In [ ]:
from obspy.core import Stream, read
from obspy.signal.trigger import coincidence_trigger
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger, trigger_onset

from pprint import pprint
import glob
import os
import sys

# Pasta com os possíveis eventos recortados 
path = "../eventos/"

if not os.path.exists(path):
        os.mkdir(path)

trigger_on = 5 # o código utiliza  
trigger_off = 2
sta = 0.5
lta = 10

############################################################
# Dia juliano
j = "063"

# Caminho para os Dados
#dados = "/home/felipe/sources/DADOS/seiscomp_NRT/2022/"
dados = "../DADOS/2022/"

# Rede e estações
rede = ['LS']
estacoes = ['JAC']
############################################################

st = Stream()
files = []
for r in rede:
    files = files + glob.glob(dados + r + "/*/*HHZ*/*" + j)

for f in files:
    for e in estacoes:
        if(e in f):
            st += read(f)

#tr = st[0].copy()
#df = tr.stats.sampling_rate
#if(df > 100):
#    tr.decimate(4)
#    df = tr.stats.sampling_rate

st.detrend()
st.filter('bandpass', freqmin=2, freqmax=20)

st2 = st.copy()
trig = coincidence_trigger("recstalta", 7, 2, st2, 2, sta=0.3, lta=15,
                          details=True)
pprint(trig)

    
for d in trig:
    
    start = d['time'] - 5
    end = start + 15
    
    pasta = path + j
    if not os.path.exists(pasta):
        os.mkdir(pasta)
    
    os.mkdir(pasta + "/" + str(d['time']))
    for s in d['stations']:
        for f in files:
            if (s in f):
                st2 = read(f)
                st += st2 
                st2.trim(start, end)
                st2.write(path + str(d['time'])+"/"+str(s), format='SAC') 
        tr = st.copy()
        tr.trim(start, end)
        tr.write(pasta  + "/" + str(d['time']) + "/" + str(s), format='SAC')
        tr.plot()
        text = input("Digite S para gravar o evento.")
                
                
                #tr.plot()

In [ ]:
estacoes

In [ ]:
dados

In [ ]:
dados + r + "/*/*HHZ*/*" + j